In [34]:
import pandas as pd
import numpy as np
import os
import re
import json 

In [3]:
#import artsy data 
cur_path = os.getcwd()
result_path = 'ArtsyResult'
file_name = 'ArtsyData.csv'
path = os.path.join(cur_path,result_path,file_name)
df = pd.read_csv(path)
# drop all incomplete 
df = df.dropna(axis=0)
df = df.reset_index(drop=True)
df.tail()

,id,media,url,artist_name,artist_url,work_name,work_year,media_long,dimension,gallery_name,gallery_url,bid,price,img_url
9354,design1494,design,https://www.artsy.net/artwork/martin-schlotz-n...,Martin Schlotz,https://www.artsy.net/artist/martin-schlotz,#2636,2017,Porcelain,14 1/5 × 4 3/10 × 4 3/10 in | 36 × 11 × 11 cm,Brutto Gusto,https://www.artsy.net/partner/brutto-gusto,False,€500,https://d7hftxdivxxvm.cloudfront.net?resize_to...
9355,design1495,design,https://www.artsy.net/artwork/eric-astoul-vase...,Eric Astoul,https://www.artsy.net/artist/eric-astoul,"Vase échancré strié, ceramic",2011,Stoneware,9 × 10 1/2 × 10 1/2 in | 22.9 × 26.7 × 26.7 cm,Maison Gerard,https://www.artsy.net/partner/maison-gerard,False,"US$3,500",https://d7hftxdivxxvm.cloudfront.net?resize_to...
9356,design1497,design,https://www.artsy.net/artwork/robert-stadler-s...,Robert Stadler,https://www.artsy.net/artist/robert-stadler,Size Matters,2022,Glazed Ceramic,66 9/10 × 15 7/10 × 15 7/10 in | 170 × 40 × 40 cm,Carwan Gallery,https://www.artsy.net/partner/carwan-gallery,False,"€10,500",https://d7hftxdivxxvm.cloudfront.net?resize_to...
9357,design1498,design,https://www.artsy.net/artwork/reinaldo-sanguin...,Reinaldo Sanguino,https://www.artsy.net/artist/reinaldo-sanguino,GAIA MOTHER VASE 5,2022,Ceramic,8 in diameter | 20.3 cm diameter,The Future Perfect,https://www.artsy.net/partner/the-future-perfect,False,"US$3,000",https://d7hftxdivxxvm.cloudfront.net?resize_to...
9358,design1499,design,https://www.artsy.net/artwork/jin-sook-so-kons...,Jin-Sook So,https://www.artsy.net/artist/jin-sook-so,Konstruktion B,2007,"Steel mesh, electroplated, silver, gold, paint...",18 3/4 × 19 3/4 × 2 1/2 in | 47.6 × 50.2 × 6.4 cm,browngrotta arts,https://www.artsy.net/partner/browngrotta-arts,False,"US$4,600",https://d7hftxdivxxvm.cloudfront.net?resize_to...


In [4]:
# To change work_year to numeric

for i in range(len(df['work_year'])):
    if not df['work_year'][i].isnumeric():
        df['work_year'][i] = df['work_year'][i][-4:]
df['work_year'] = df['work_year'].astype(int)

/var/folders/2p/2p_g91q95gj9px5nbxvkbw3h0000gn/T/ipykernel_29081/4207513392.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['work_year'][i] = df['work_year'][i][-4:]


In [5]:
usd, pound, eur = "$", "£", "€"
#helpers
def toUsd(currency, amount):
    if currency == "pound":
        return amount*1.21
    if currency == "eur":
        return amount*1.04
    else:
        return amount

def getPrice (price):
    if not isinstance(price, str):
        return price
    if '.' in price:
        print("error:", price)
        return
    if pound in price:
        currency = "pound"
    if eur in price:
        currency = "eur"
    else:
        currency = "usd"
    price_list = price.split("–")
    if len(price_list) > 2:
        print("error: ",price_list)
        return
    if len(price_list) == 1:
        amount =  ''.join(c for c in price if c.isdigit())
        return toUsd(currency,int(amount))
    else:
        amount1 =  ''.join(c for c in price_list[0] if c.isdigit())
        amount2 =  ''.join(c for c in price_list[1] if c.isdigit())
        amount = (int(amount1)+ int(amount2))//2
        return toUsd(currency,amount)
    


In [6]:
# observation: all prices are in unit usd, gbp or eur
usd, pound, eur = "$", "£", "€"
# To change price to number; estimated price to an actual number
count = 0
for i in range(len(df['price'])):
    try:
        df['price'][i] = getPrice(df['price'][i])
    except:
        df.drop(i, inplace= True)
df = df.reset_index(drop=True)
df  



/var/folders/2p/2p_g91q95gj9px5nbxvkbw3h0000gn/T/ipykernel_29081/218915616.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'][i] = getPrice(df['price'][i])
/var/folders/2p/2p_g91q95gj9px5nbxvkbw3h0000gn/T/ipykernel_29081/218915616.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'][i] = getPrice(df['price'][i])
/var/folders/2p/2p_g91q95gj9px5nbxvkbw3h0000gn/T/ipykernel_29081/218915616.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

,id,media,url,artist_name,artist_url,work_name,work_year,media_long,dimension,gallery_name,gallery_url,bid,price,img_url
0,painting1,painting,https://www.artsy.net/artwork/jeremy-mann-the-...,Jeremy Mann,https://www.artsy.net/artist/jeremy-mann,The Spanish Beach #2,2022,Oil on panel,4 × 5 in | 10.2 × 12.7 cm,Principle Gallery,https://www.artsy.net/partner/principle-gallery,False,1850,https://d7hftxdivxxvm.cloudfront.net?resize_to...
1,painting2,painting,https://www.artsy.net/artwork/jeremy-mann-the-...,Jeremy Mann,https://www.artsy.net/artist/jeremy-mann,The Spanish Beach #1,2022,Oil on panel,4 × 5 in | 10.2 × 12.7 cm,Principle Gallery,https://www.artsy.net/partner/principle-gallery,False,1850,https://d7hftxdivxxvm.cloudfront.net?resize_to...
2,painting3,painting,https://www.artsy.net/artwork/koen-delaere-unt...,Koen Delaere,https://www.artsy.net/artist/koen-delaere,Untitled (Pistachio and Red),2015,Oil and acrylic on linen,70 4/5 × 49 1/5 in | 179.8 × 125 cm,NAVA Contemporary,https://www.artsy.net/partner/nava-contemporary,False,12025,https://d7hftxdivxxvm.cloudfront.net?resize_to...
3,painting4,painting,https://www.artsy.net/artwork/jeremy-mann-sliver,Jeremy Mann,https://www.artsy.net/artist/jeremy-mann,Sliver,2019,Oil on panel,49 × 26 in | 124.5 × 66 cm,Principle Gallery,https://www.artsy.net/partner/principle-gallery,False,21500,https://d7hftxdivxxvm.cloudfront.net?resize_to...
4,painting5,painting,https://www.artsy.net/artwork/koen-delaere-unt...,Koen Delaere,https://www.artsy.net/artist/koen-delaere,"Untitled (Red, Yellow and Blue)",2015,"Oil, cyanotype, oil and acrylic on linen",62 × 39 3/10 in | 157.5 × 99.8 cm,NAVA Contemporary,https://www.artsy.net/partner/nava-contemporary,False,10525,https://d7hftxdivxxvm.cloudfront.net?resize_to...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9335,design1494,design,https://www.artsy.net/artwork/martin-schlotz-n...,Martin Schlotz,https://www.artsy.net/artist/martin-schlotz,#2636,2017,Porcelain,14 1/5 × 4 3/10 × 4 3/10 in | 36 × 11 × 11 cm,Brutto Gusto,https://www.artsy.net/partner/brutto-gusto,False,520.0,https://d7hftxdivxxvm.cloudfront.net?resize_to...
9336,design1495,design,https://www.artsy.net/artwork/eric-astoul-vase...,Eric Astoul,https://www.artsy.net/artist/eric-astoul,"Vase échancré strié, ceramic",2011,Stoneware,9 × 10 1/2 × 10 1/2 in | 22.9 × 26.7 × 26.7 cm,Maison Gerard,https://www.artsy.net/partner/maison-gerard,False,3500,https://d7hftxdivxxvm.cloudfront.net?resize_to...
9337,design1497,design,https://www.artsy.net/artwork/robert-stadler-s...,Robert Stadler,https://www.artsy.net/artist/robert-stadler,Size Matters,2022,Glazed Ceramic,66 9/10 × 15 7/10 × 15 7/10 in | 170 × 40 × 40 cm,Carwan Gallery,https://www.artsy.net/partner/carwan-gallery,False,10920.0,https://d7hftxdivxxvm.cloudfront.net?resize_to...
9338,design1498,design,https://www.artsy.net/artwork/reinaldo-sanguin...,Reinaldo Sanguino,https://www.artsy.net/artist/reinaldo-sanguino,GAIA MOTHER VASE 5,2022,Ceramic,8 in diameter | 20.3 cm diameter,The Future Perfect,https://www.artsy.net/partner/the-future-perfect,False,3000,https://d7hftxdivxxvm.cloudfront.net?resize_to...


In [7]:
clean_file_name = 'CleanedArtsyData.csv'
filepath = path = os.path.join(cur_path, result_path, clean_file_name)
df.to_csv(filepath, index=False)


### If needed, will change the format for dimension to numeric

plan:

will replace the dimension column to 3 columns (length, wide, height), unit: cm
will add one column "three_d" type: boolean, to indicate if it is a 3 dimension work
height will be 0 for non 3 dimensional works

Raw data format examples: 
16 × 16 × 17 1/2 in | 40.6 × 40.6 × 44.5 cm
144 in diameter | 365.8 cm diameter
7 × 11 × 7 1/2 in | 17.8 × 27.9 × 19.1 cm

In [27]:
# get artist name list
#artist_set = set(df['artist_name'])
artist_set = set()
artist_artid_pair = {}
for i in range(len(df)):
    if df['artist_name'][i] in artist_set:
        artist_artid_pair[df['artist_name'][i]].append(df['id'][i])
    else:
        artist_set.add(df['artist_name'][i])
        artist_artid_pair[df['artist_name'][i]] = [df['id'][i]]



In [39]:
#create artist dataframe
artist_list = list(artist_set)
artist_df = pd.DataFrame(artist_list)
artist_df.reset_index(inplace=True)
artist_df = artist_df.rename(columns = {'index':'id'})
artist_df = artist_df.rename(columns = {0:'artist_name'})
artist_df

,id,artist_name
0,0,Lori Cozen-Geller
1,1,Ed van der Elsken
2,2,Minegishi Seikō
3,3,Brittany Fanning
4,4,Frances Goodman
...,...,...
2861,2861,Morley
2862,2862,George Barris
2863,2863,Balint Zsako
2864,2864,Charlie Bluett


In [41]:
#save the artist data
artist_file_name = 'ArtsyArtist.csv'
filepath = path = os.path.join(cur_path, result_path, artist_file_name)
artist_df.to_csv(filepath, index=False)

In [42]:
# save artist_artid_pair
artist_artid_pair_file_name = "ArtistWork.json"
#json_object = json.dumps(artist_artid_pair)
#gqm is tired ...
filepath = os.path.join(cur_path, result_path, artist_artid_pair_file_name)
with open(filepath, 'w') as f:
    json.dump(artist_artid_pair, f)

In [43]:
artist_artid_pair

{'Jeremy Mann': ['painting1',
  'painting2',
  'painting4',
  'painting12',
  'painting19',
  'painting22',
  'painting26',
  'painting31',
  'painting59',
  'painting61',
  'painting79',
  'painting87',
  'painting506',
  'painting530',
  'painting609',
  'painting692',
  'painting749',
  'painting766',
  'painting772',
  'painting886',
  'painting923',
  'painting1152',
  'works-on-paper15',
  'works-on-paper28'],
 'Koen Delaere': ['painting3',
  'painting5',
  'painting55',
  'mixed-media31',
  'mixed-media34',
  'mixed-media81'],
 'Teodora Axente': ['painting6',
  'painting7',
  'painting13',
  'painting14',
  'painting17',
  'painting27',
  'painting536'],
 'Gigi Mills': ['painting8', 'painting367'],
 'Pierre Bergian': ['painting9',
  'painting114',
  'painting126',
  'painting142',
  'painting180',
  'painting463',
  'painting806'],
 'Robert Standish': ['painting10'],
 'Raphael Adjetey Adjei Mayne': ['painting11',
  'mixed-media6',
  'mixed-media19',
  'mixed-media30',
  'mixed-m